In [1]:
import gym
import copy
import numpy as np
import time as time
import torch
import torch.nn as nn
import gym_flappyBird
import genetics as gen
from tensorboardX import SummaryWriter
import matplotlib.pyplot as plt
import math
import pygame
from pygame.locals import *

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


# Maschine

## Feature extrahieren

<img src="sprites/ingame.jpg" align="left" >

In [ ]:
def generateFeatures(state):
    bird = state['bird']
    #print(type(bird))
    posY = bird.Y
    speedY = bird.speedY
    pipes = state['pipes']
    #print(pipes[0].pos)
    return posY, speedY, pipes[0].pos,pipes[0].height,pipes[0].gap

## Reward  berechnen

<img src="sprites/ingame_2.jpg" align="left" >

In [ ]:
def computeReward(state_old, state_new):
    return 0

# Aktion festlegen

In [ ]:
def birdAction(decission, bird):
        bird.force = [0.0, 50*decission[0]]

# Mutationsparameter

<img src="sprites/ingame_3.jpg" align="left" >

In [ ]:
NOISE_STD = 0.5
POPULATION_SIZE = 250
PARENTS_COUNT = 15

# Neunales Netz 

## Layer
nn.Linear(X, Y), Input dim X, Output dim Y

## Aktivierungsfunktionen
nn.ReLU()


nn.Sigmoid()


nn.Softmax(dim=1))



## Topologie
nn.Sequential(Layer, Aktivierungsfunktion, Layer, Aktivierungsfunktion...., Layer,Softmax)
###### Beispiel
nn.Sequential(nn.Linear(5, 5),nn.Softmax(dim=1))

In [2]:
net = nn.Sequential(nn.Linear(5, 5),nn.Sigmoid(),nn.Linear(5, 5),nn.Sigmoid(), nn.Linear(5, 1))

In [3]:
#net = nn.Sequential(nn.Linear(5, 5),nn.ReLU(),nn.Linear(5, 2),nn.Softmax(dim=1))

In [4]:
for name, param in net.named_parameters():
    print(name, param)

0.weight Parameter containing:
tensor([[ 0.0818, -0.3067, -0.3164,  0.3095, -0.0179],
        [ 0.3950,  0.2697, -0.2953, -0.4458, -0.1279],
        [-0.2016, -0.2666, -0.0030,  0.1808, -0.2403],
        [ 0.4021, -0.3385, -0.0999,  0.4189, -0.1962],
        [-0.0852,  0.0740,  0.0865, -0.1790,  0.1862]], requires_grad=True)
0.bias Parameter containing:
tensor([-0.0036,  0.3702,  0.4280, -0.3133, -0.1822], requires_grad=True)
2.weight Parameter containing:
tensor([[-0.0063,  0.4080, -0.3806, -0.1752, -0.3895],
        [ 0.1459, -0.2627,  0.3718,  0.3536,  0.1857],
        [-0.0603, -0.1330,  0.1552, -0.4410,  0.2385],
        [ 0.0128,  0.2480, -0.2938,  0.2682, -0.2566],
        [ 0.4353, -0.1976, -0.3321,  0.4292,  0.1503]], requires_grad=True)
2.bias Parameter containing:
tensor([ 0.0511,  0.1929, -0.3110, -0.2468,  0.3283], requires_grad=True)
4.weight Parameter containing:
tensor([[-0.3348, -0.4331,  0.1173, -0.0624,  0.2514]], requires_grad=True)
4.bias Parameter containing:
tens

## [ Abstand zwischen,  Höhe der Pipes, Grösse der Lücke

In [5]:
Interval_distance = [250, 350]
Interval_height = [100,300]
Interval_gap = [120,130]

## Max Score

In [ ]:
Score_Max = 4000

In [ ]:
fittestBirds = []

env = gym.make("scienceCampBird-v1")
env.setPipeIntervals([Interval_distance, Interval_height,Interval_gap])
population = gen.Population(POPULATION_SIZE, 5, 2, computeReward, net)
env.setAction(birdAction)
  #  print(len(population.nets))
population.evaluate_on_env(env,generateFeatures, Score_Max)
ecount = 0  
while True:
    population = gen.mutate_population(population, PARENTS_COUNT, NOISE_STD)
    population.evaluate_on_env(env, generateFeatures, Score_Max)
    fittestBirds.append(population.population[0])
    ecount +=1
    if(ecount % 20 == 0):
        net = population.population[0][1]
        score_e = population.population[0][0]
        score_p = env.playWithNet(net, generateFeatures, Score_Max, computeReward, ecount)
        print("----------------------------------------------------------------------------")
        print("----------------------------------------------------------------------------")
        print('Population: ', ecount)
        print("____________________________________________________________________________")
        print('Score Training: ', score_e, ' Score Spiel: ', score_p)
        print("____________________________________________________________________________")
        for name, param in net.named_parameters():
            print(name, param)

# Plott

In [ ]:
scores = [score[0] for score in fittestBirds]
fig, ax = plt.subplots()
ax.plot(scores)

ax.set(xlabel='Pupulation', ylabel='Score',
       title=' Scores der besten Vogels der Population')
ax.grid()

fig.savefig("test.png")
plt.show()

In [ ]:
i = 0
for bird in fittestBirds:
        i +=1
        net = bird[1]
        score_p = bird[0]
        print("----------------------------------------------------------------------------")
        print("----------------------------------------------------------------------------")
        print('Population: ', i)
        print("____________________________________________________________________________")
        print('Score: ', score_p)
        print("____________________________________________________________________________")
        for name, param in net.named_parameters():
            print(name, param)